In [1]:
import sys

%load_ext autoreload
%autoreload 2

sys.path.append("..")

from src.make_predictions import *
from src.data.process_data import *

import sqlite3
import joblib
import pandas as pd

In [2]:
final_stacked_model = joblib.load('../models/finalized_stacked_model')

prev_season_elo = pd.read_csv("../data/clean/final_elo_ratings_2020.csv")

In [3]:
connection = sqlite3.connect('../data/nba.db')

df = load_current_season_team_data(connection, 2021)



In [4]:
home_team = 'PHI'
away_team = 'LAC'
home_spread = -5
away_spread = -home_spread
home_ml = 1.51
away_ml = 2.7

conn = sqlite3.connect('../data/nba.db')
season = 2021

df = load_current_season_team_data(conn, season)

df = clean_team_data(df)

df = prep_for_aggregation(df)

spreads, moneylines = load_current_season_betting_data(conn, 2021)
clean_moneylines = clean_moneyline_df(df = moneylines)
clean_spreads = clean_spreads_df(df = spreads)

full_df = merge_betting_and_boxscore_data(clean_spreads, clean_moneylines, df)
full_df = create_matchups(full_df)
full_df = get_team_and_opp_ewm(full_df, min_periods=1, alpha=0.1)

full_df = add_percentage_features(full_df)
full_df = add_rest_days_for_model(full_df)

current_elo, full_df = get_current_season_elo_ratings(full_df, prev_season_elo)

matchup = create_matchup_rows_for_model(full_df, home_team, away_team,
                                                home_spread, away_spread, home_ml, away_ml)

print("nulls:", matchup.isnull().sum().sum())
matchup

Progress: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.16it/s]
progress:: 1408it [00:00, 5083.20it/s]


nulls: 0


,RECORD_team_diff,FG2M_team_diff,FG2A_team_diff,FG3M_team_diff,FG3A_team_diff,FTM_team_diff,FTA_team_diff,OREB_team_diff,DREB_team_diff,REB_team_diff,...,AST_RATIO_team_diff,AST_RATIO_opp_diff,TOV_PCT_team_diff,TOV_PCT_opp_diff,team_elo_pred,team_rating_i,opp_rating_i,elo_MOV_pred,SPREAD_team,ML_team
0,0.235933,1.527986,1.502465,-1.135059,-3.458006,4.389851,5.001732,-0.793731,-0.081454,-0.875185,...,0.010349,-2.304419,-1.104695,1.808602,0.734147,1596.24496,1488.78857,6.3,-5.0,1.51
1,-0.235933,-1.527986,-1.502465,1.135059,3.458006,-4.389851,-5.001732,0.793731,0.081454,0.875185,...,-0.010349,2.304419,1.104695,-1.808602,0.265853,1488.78857,1596.24496,-6.3,5.0,2.70


In [5]:
connection = sqlite3.connect('../data/nba.db')

# on = sqlite3.connect("data.db")
# cursor = connection.cursor()

# cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
# print(cursor.fetchall())

# connection.close()

df = load_current_season_team_data(connection, season=2021)
df

,SEASON,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,PCT_PTS_FB,PCT_PTS_FT,PCT_PTS_OFF_TOV,PCT_PTS_PAINT,PCT_AST_2PM,PCT_UAST_2PM,PCT_AST_3PM,PCT_UAST_3PM,PCT_AST_FGM,PCT_UAST_FGM
53676,2021-22,1610612747,LAL,Los Angeles Lakers,0022100002,2021-10-19,LAL vs. GSW,L,240,45,...,0.175,0.079,0.167,0.298,0.367,0.633,0.667,0.333,0.467,0.533
53677,2021-22,1610612744,GSW,Golden State Warriors,0022100002,2021-10-19,GSW @ LAL,W,240,41,...,0.083,0.207,0.165,0.380,0.630,0.370,0.929,0.071,0.732,0.268
53678,2021-22,1610612749,MIL,Milwaukee Bucks,0022100001,2021-10-19,MIL vs. BKN,W,240,48,...,0.165,0.110,0.173,0.331,0.323,0.677,0.882,0.118,0.521,0.479
53679,2021-22,1610612751,BKN,Brooklyn Nets,0022100001,2021-10-19,BKN @ MIL,L,240,37,...,0.144,0.125,0.019,0.327,0.450,0.550,0.588,0.412,0.514,0.486
53680,2021-22,1610612766,CHA,Charlotte Hornets,0022100003,2021-10-20,CHA vs. IND,W,240,46,...,0.122,0.146,0.179,0.439,0.545,0.455,0.846,0.154,0.630,0.370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55079,2021-22,1610612741,CHI,Chicago Bulls,0022100730,2022-01-24,CHI @ OKC,W,240,41,...,0.126,0.126,0.144,0.342,0.692,0.308,0.867,0.133,0.756,0.244
55080,2021-22,1610612756,PHX,Phoenix Suns,0022100713,2022-01-24,PHX vs. UTA,W,240,47,...,0.061,0.070,0.148,0.452,0.471,0.529,0.923,0.077,0.596,0.404
55081,2021-22,1610612762,UTA,Utah Jazz,0022100713,2022-01-24,UTA @ PHX,L,240,40,...,0.101,0.138,0.147,0.404,0.346,0.654,0.500,0.500,0.400,0.600
55082,2021-22,1610612754,IND,Indiana Pacers,0022100712,2022-01-24,IND @ NOP,L,240,38,...,0.044,0.159,0.142,0.283,0.474,0.526,0.895,0.105,0.684,0.316


In [6]:
connection = sqlite3.connect('../data/nba.db')


main(connection, 2021, final_stacked_model, prev_season_elo, append=True)

Progress: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.19it/s]
progress:: 1446it [00:00, 5264.30it/s]


,game_date,home_team,away_team,home_spread,home_cover_probs,away_cover_probs,home_cover_avg
0,2022-01-27,PHI,LAL,-4.0,0.496490,0.502261,0.497114
1,2022-01-27,GSW,MIN,-6.0,0.514649,0.484663,0.514993


In [7]:
final_stacked_model_sfs = joblib.load('../models/finalized_stacked_model_retrain-21-12-20')

connection = sqlite3.connect('../data/nba.db')


main(connection, 2021, final_stacked_model_sfs, prev_season_elo, append=False)


Progress: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.20it/s]
progress:: 1446it [00:00, 5315.33it/s]


,game_date,home_team,away_team,home_spread,home_cover_probs,away_cover_probs,home_cover_avg
0,2022-01-27,PHI,LAL,-4.0,0.527386,0.479824,0.523781
1,2022-01-27,GSW,MIN,-6.0,0.533777,0.468613,0.532582


In [8]:
final_stacked_model_v2 = joblib.load('../models/final_stacked_clf_v2-21-12-20')

connection = sqlite3.connect('../data/nba.db')

main(connection, 2021, final_stacked_model_v2, prev_season_elo, append=False)


updating basic team boxscores
updating advanced team boxscores
num_games_updated: 0
All team advanced boxscores up to date in season 2021-22
updating scoring boxscores
num_games_updated: 0
All team advanced boxscores up to date in season 2021-22
updating moneyline data
moneyline table is up to date
updating spreads data
Updating spreads for 0 days
spreads table is up to date


Progress: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.19it/s]
progress:: 1446it [00:00, 5445.33it/s]


,game_date,home_team,away_team,home_spread,home_cover_probs,away_cover_probs,home_cover_avg
0,2022-01-27,PHI,LAL,-4.0,0.498496,0.501641,0.498427
1,2022-01-27,GSW,MIN,-6.0,0.518017,0.481770,0.518123
